In [1]:
import pandas as pd
import numpy as np

# Готовим данные

In [2]:
df = pd.read_csv('feedback.csv', index_col=None, header=None, escapechar='\\')

In [3]:
df.fillna(' ', inplace=True)

In [4]:
review = pd.DataFrame()

In [5]:
review['target'] = df[0]

In [6]:
review['text'] = df.apply(lambda x: ' '.join(x[4:]), axis=1)

In [7]:
del(df)

# Начинаем построение модели

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor

from keras.layers import Dense
from keras.models import Sequential
from keras.callbacks import EarlyStopping, BaseLogger

Using Theano backend.
/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-09-10 23:12:22.693016. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


## Разобьем и векторизуем данные

In [9]:
x_train, x_test, y_train, y_test = train_test_split(review.text, review.target, test_size=0.2)
x_train, x_eval, y_train, y_eval = train_test_split(x_train, y_train, test_size=0.1)

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-09-10 23:12:24.474824. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [10]:
vectorizer = TfidfVectorizer(analyzer='word', ngram_range=(1,2), max_df=0.9, min_df=0.001)

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-09-10 23:12:24.547874. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [11]:
train_data = vectorizer.fit_transform(x_train)
eval_data = vectorizer.transform(x_eval)
test_data = vectorizer.transform(x_test)

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-09-10 23:12:24.555714. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


### Линейный классификатор

In [12]:
model = Ridge(alpha=0.01, fit_intercept=True, random_state=42, max_iter=200)

/usr/lib/python3.6/json/encoder.py:199: DeprecationWarning: Interpreting naive datetime as local 2017-09-10 23:12:30.489612. Please add timezone info to timestamps.
  chunks = self.iterencode(o, _one_shot=True)


In [13]:
model.fit(train_data, y_train)

/usr/lib/python3.6/site-packages/sklearn/linear_model/sag.py:286: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  "the coef_ did not converge", ConvergenceWarning)


Ridge(alpha=0.01, copy_X=True, fit_intercept=True, max_iter=200,
   normalize=False, random_state=42, solver='auto', tol=0.001)

### Градиентный лес

In [14]:
tree_model = XGBRegressor(random_state=42, colsample_bytree=0.5, reg_alpha=0.1)

In [15]:
tree_model.fit(train_data, y_train, eval_metric=mean_squared_error)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=0.5, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:linear', random_state=42,
       reg_alpha=0.1, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

### Нейронная сеть

In [16]:
neural_model = Sequential(layers=[
    Dense(64, input_shape=(train_data.shape[1], )),
    Dense(32, activation='tanh'),
    Dense(16, activation='relu'),
    Dense(1, activation='linear')
])

In [17]:
neural_model.compile('sgd', 'mse', metrics=['accuracy'])

In [18]:
neural_model.fit(train_data.toarray(), y_train, epochs=100, validation_split=0.05, batch_size=128,
                 callbacks=[EarlyStopping(min_delta=1e-3, patience=5), BaseLogger()])

Train on 12543 samples, validate on 661 samples
Epoch 1/100
12543/12543 [==============================] - 20s - loss: 2.8297 - acc: 0.1533 - val_loss: 1.6133 - val_acc: 0.1710

## Сделаем ансамбль из полученных моделей

In [19]:
eval_pred = list()
eval_pred.append(model.predict(eval_data))
eval_pred.append(tree_model.predict(eval_data))
eval_pred.append(neural_model.predict(eval_data.toarray()).reshape(eval_pred[1].shape))
eval_pred = np.array(eval_pred)

In [20]:
test_pred = list()
test_pred.append(model.predict(test_data))
test_pred.append(tree_model.predict(test_data))
test_pred.append(neural_model.predict(test_data.toarray()).reshape(test_pred[1].shape))
test_pred = np.array(test_pred)

In [26]:
eval_model = LinearRegression()
eval_model.fit(eval_pred.T, y_eval)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

Качество на отложенной выборке:

In [24]:
print(mean_squared_error(y_eval, model.predict(eval_data)))
print(mean_squared_error(y_eval, tree_model.predict(eval_data)))
print(mean_squared_error(y_eval, neural_model.predict(eval_data.toarray()).reshape(eval_pred[1].shape)))

1.44731314682
1.2155146793
0.913263742836


In [27]:
print(mean_squared_error(y_test, eval_model.predict(test_pred.T)))
print(mean_squared_error(y_test, test_pred.mean(axis=0)))

0.764441853501
0.84731283657
